
## GiftGeniusAgent——你的送礼智能Agent

### 项目简介
本项目演示一个基于HelloAgents框架的智能送礼Agent

### 作者信息
- 姓名：张善祺
- GitHub:@jack6249
- 日期：2025-11-21


### 第1部分：环境配置

In [ ]:
#导入库和参数配置
from hello_agents import SimpleAgent, HelloAgentsLLM, ReflectionAgent
from hello_agents.tools import Tool, ToolParameter
from typing import Dict, Any, List
from tavily import TavilyClient
import os
import json
import re
import numpy as np 
from dotenv import load_dotenv

load_dotenv()

#LLM参数
LLM_MODEL_ID = os.getenv("LLM_MODEL_ID")
#"deepseek-chat"
LLM_API_KEY = os.getenv("LLM_API_KEY")
LLM_BASE_URL = os.getenv("LLM_BASE_URL")
#Tavily参数
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

print("✅ 环境配置完成")


### 第2部分：定义工具

In [ ]:
class BatchSearchTool(Tool):
    def __init__(self):
        super().__init__(
            name="batch_search",
            description="高级批量搜索工具。"
        )

    def search_raw(self, query: str) -> List[Dict]:
        if "TAVILY_API_KEY" not in os.environ: 
            print("❌ 错误：缺少 API Key")
            return []

        print(f"    🚀 [直连搜索] 正在抓取: {query} ...")
        try:
            tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
            # 搜索包含图片
            response = tavily.search(query, max_results=5, include_images=True) # 增加到5条，提高命中率
            
            results = []
            # 1. 提取文本结果
            if 'results' in response:
                for r in response['results']:
                    results.append({
                        "title": r['title'],
                        "url": r['url'],
                        "content": r['content'], 
                        "type": "text"
                    })
            
            # 2. 提取图片结果
            if 'images' in response and response['images']:
                results.append({
                    "images": response['images'][:3], # 取前3张
                    "type": "image"
                })
                
            return results
            
        except Exception as e:
            print(f"      ⚠️ 搜索异常: {e}")
            return []
    
    # 兼容 Agent 调用接口
    def run(self, parameters: Any) -> str:
        return "请使用 Python 代码直接调用 search_raw 方法获取数据。"
    
    def get_parameters(self) -> List[ToolParameter]:
        return [ToolParameter(name="query", type="string", description="关键词", required=True)]

print("✅ BatchSearchTool定义完成")

### 第3部分：创建智能体



In [ ]:
from hello_agents import ToolRegistry

#创建工具
tool_registry = ToolRegistry()
tool_registry.register_tool(BatchSearchTool())

print(tool_registry.list_tools())


# 初始化大模型
llm = HelloAgentsLLM()

# --- 1. 军师 (Profiler) - 已升级支持多维度画像 ---
PROFILER_PROMPT = """
你是一个精通 MBTI 人格分析与消费市场趋势的 "送礼军师"。
你的任务是根据用户提供的多维度画像，制定 3 个**极度精准**的搜索关键词。

【⚠️ 时效性死命令 (CRITICAL)】
当前时间视作 **2025年11月**。
1. **严禁过时**：绝对不要推荐 2024 年或更早的旧款（除非是经典恒久款如黑胶唱片）。
2. **锁定新品**：对于**化妆品、数码、盲盒**，必须搜索 **"2025圣诞限定"**、**"2025秋冬新品"** 或 **"2026春季预告"**。
3. **价格尺度**：给出的价格单位是人民币元。请严格遵照范围进行联想，禁止超出预算范围。

为了确保推荐质量，请参考以下的【优秀思考范例】：

### 范例 1
**用户画像**: 
- 女, 26岁, ISFP (探险家), 金牛座
- 预算: 500-1000元
- 场景: 情人节
- 自定义: 喜欢有质感的生活小物
**军师分析**: 
ISFP 重视审美和感官体验，金牛座喜欢实实在在的质感。情人节需要浪漫。
**生成策略**:
1. 观夏 (To Summer) 昆仑煮雪 晶石香薰 (符合质感与审美)
2. 野兽派 2025 情人节限定 睡衣礼盒 (金牛座喜欢的舒适)
3. 富士 Instax mini Evo 拍立得 (记录生活瞬间)

### 范例 2
**用户画像**: 
- 男, 30岁, INTJ (建筑师), 处女座
- 预算: 1000元以上
- 场景: 生日
- 自定义: 程序员，喜欢整洁
**军师分析**: 
INTJ 追求极致的逻辑和效率，处女座有洁癖，喜欢桌面整洁。
**生成策略**:
1. Keychron Q1 Pro 机械键盘 铝坨坨 (符合极客对工具的追求)
2. 明基 (BenQ) ScreenBar Halo 屏幕挂灯 (极致护眼与桌面美学)
3. 赫曼米勒 (Herman Miller) 显示器支架 (人体工学)

### 范例 3
**用户画像**: 
- 女, 20岁, ENFP (竞选者), 狮子座
- 预算: 300元以内
- 场景: 圣诞节
- 自定义: 喜欢二次元，痛包
**军师分析**: 
ENFP 热情奔放，狮子座喜欢张扬、闪亮的东西。预算有限但要素多。
**生成策略**:
1. 泡泡玛特 圣诞系列 盲盒整端 (符合节日气氛和二次元)
2. WEGO 痛包 镭射款 (符合自定义需求，狮子座喜欢的亮眼)
3.  Chiikawa 吉伊卡哇 圣诞公仔 (当下顶流二次元IP)

---

**现在的任务**：
请根据以下【当前用户画像】进行分析，模仿上述范例的深度，制定搜索策略。

【当前用户画像】
{user_profile_text}

【关键词生成要求】
1. **必须具体**：格式为 `[品牌] + [产品名/系列] + [限定/属性]`。
2. **拒绝大词**：严禁搜索 "礼物"、"口红"、"玩具" 这种泛词。
3. **必须包含品牌**：根据预算推断合适的品牌（如：预算低选名创优品/泡泡玛特，预算高选Dior/索尼）。

【输出格式】
只输出 3 行搜索关键词，每行一个。不要输出分析过程，不要序号。
"""

profiler_agent = SimpleAgent(
    llm=llm,
    name="Agent_Profiler",
    system_prompt=PROFILER_PROMPT
)

# ==============================================================================
# 2. 提取专家 (Extractor) - 数据清洗 (加入反思机制)
# ==============================================================================
EXTRACTOR_PROMPTS = {
    "initial": """
    你是一个 **纯粹的数据提取器**。
    你的任务是从 `<<<原始数据>>>` 中提取商品参数。

    ### 🧠 CoT 提取逻辑
    1.  **扫描**: 快速浏览文本，寻找 "¥", "$", "http" 等关键符号。
    2.  **验证**: 确认找到的信息是否属于"核心商品"（而非广告推荐）。
    3.  **格式化**: 将提取到的信息填入 JSON。

    ### 🚫 绝对禁令
    1.  **严禁伪造**: 没找到价格就填 "暂无报价"，绝对不要编数字！
    2.  **严禁 API 格式**: 不要输出 `{{ "status": ... }}`。
    3.  **严禁 Markdown**: 直接输出列表 `[...]`。

    ### ✅ 正确范例
    输入: ...Dior 口红...价格 ¥350...图片 http://img...
    输出:
    [
        {{
            "name": "Dior 口红",
            "price": "¥350",
            "img": "http://img..."
        }}
    ]

    ---
    <<<原始数据开始>>>
    {task}
    <<<原始数据结束>>>
 
    请提取数据并输出 JSON 列表：
    """,
    "reflect": "检查输出：是否以 `[` 开头？是否包含幻觉商品？",
    "refine": "修正格式，只输出纯净的 JSON 列表 `[...]`。"
}

extractor_agent = ReflectionAgent(
    name="Agent_Extractor",
    llm=llm,
    max_iterations=1, 
    custom_prompts=EXTRACTOR_PROMPTS
)

# ==============================================================================
# 3. 种草达人 (Pitcher) - 文案创作 (加入风格指导)
# ==============================================================================
PITCHER_PROMPT = """
你是一个 **金牌种草文案**。
用户会给你一个 **【商品名称】**。

### 🎯 关键要点 (Few Points)
1.  **痛点直击**: 一句话说清楚为什么买它（限定？显白？绝美？）。
2.  **情绪价值**: 使用 "绝绝子", "氛围感", "心动" 等高频热词。
3.  **字数限制**: 严格控制在 **40字以内**，短小精悍。
4.  **Emoji**: 必须包含 1-2 个 emoji。

### 🌟 创作范例 (Few-Shot)
**输入**: Dior 999 烈艳蓝金
**输出**: 💄本宫不死终是妃！Dior 999 传奇正红，显白更有气场，送女友绝对没错！

**输入**: 泡泡玛特 Labubu 坐坐派对
**输出**: ✨太可爱了吧！Labubu 坐坐派对系列，每一个都丑萌到心巴上，摆在桌上超治愈~

**输入**: 罗技 MX Master 3S
**输出**: 🖱️打工人本命！罗技 Master 3S 静音又顺滑，人体工学设计，手腕再也不累了。

---

**当前任务**：
请为【{input}】写一句朋友圈风格种草语。
"""
pitcher_agent = SimpleAgent(
    llm=llm,
    name="Agent_Pitcher",
    system_prompt=PITCHER_PROMPT
)

print("✅ 智能体初始化完成！")

### 第4部分:读取数据

In [ ]:

INPUT_FILENAME = "data/test_cases.json"

def load_user_profile(filename):
    # 1. 检查文件是否存在
    if not os.path.exists(filename):
        print(f"⚠️ 未找到配置文件: {filename}")
        # 如果没有文件，将默认数据写入文件
        default_data = {
            "性别": "女",
            "年龄": "24岁",
            "MBTI": "ENFP",
            "星座": "天秤座",
            "预算": "500元以内",
            "节日": "恋爱一周年纪念日",
            "自定义": "喜欢二次元，平时喜欢喝咖啡，不要送太实用的家电"
        }
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(default_data, f, ensure_ascii=False, indent=4)
        print(f"✅ 已自动生成默认配置文件，请修改 {filename} 后再次运行。")
        return default_data

    # 2. 读取文件内容
    try:
        with open(filename, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"✅ 成功加载用户画像: {filename}")
        print(f"📋 内容预览: {json.dumps(data, ensure_ascii=False)}")
        return data
    except Exception as e:
        print(f"❌ 读取 JSON 失败: {e}")
        return {}

# 加载数据
user_input_data = load_user_profile(INPUT_FILENAME)




### 第5部分：生成礼物计划

In [ ]:

def parse_budget_range(budget_str):
    """解析用户预算字符串，返回 (min, max)"""
    nums = [float(x) for x in re.findall(r'\d+', str(budget_str).replace(',', ''))]
    if not nums: return 0, 999999 
    if "以内" in budget_str or "以下" in budget_str: return 0, nums[0]
    if "以上" in budget_str: return nums[0], 999999
    if len(nums) >= 2: return min(nums), max(nums)
    return 0, nums[0]

def extract_all_prices(raw_results):
    """从搜索结果列表中提取所有有效的价格"""
    prices = []
    for res in raw_results:
        # 只处理文本类型的结果
        if res.get('type') == 'text':
            text = res.get('title', '') + " " + res.get('content', '')
            # 匹配 ¥, $, 元 等格式
            matches = re.findall(r'(?:¥|￥|\$|HK\$|NT\$)\s*(\d+(?:,\d{3})*(?:\.\d+)?)', text)
            for m in matches:
                val = float(m.replace(',', ''))
                # 过滤掉像年份(2025)或过小/过大的异常值
                if 10 < val < 100000 and val not in [2024, 2025, 2026]:
                    prices.append(val)
            # 备用正则：匹配 "xxx元"
            matches_yuan = re.findall(r'(\d+(?:,\d{3})*(?:\.\d+)?)\s*元', text)
            for m in matches_yuan:
                val = float(m.replace(',', ''))
                if 10 < val < 100000 and val not in [2024, 2025, 2026]:
                    prices.append(val)
    return prices

def find_best_product(hunter, profiler_agent, keyword, budget_limit):
    """
    智能搜索核心函数：包含 初次搜索 -> 价格校验 -> 策略修正 -> 自动降级 -> 智能兜底
    """
    # 容器：用于收集所有搜索到的潜在结果（用于兜底）
    all_candidates = []
    current_kw = keyword
    
    # --- Round 1: 首次搜索 ---
    print(f"       🕵️ 第1次搜索: {current_kw} 价格")
    results_1 = hunter.search_raw(f"{current_kw} 价格 RMB")
    
    # 提取第一轮搜索到的通用图片（以防具体条目没图）
    fallback_img = ""
    for r in results_1:
        if r.get('images'): 
            fallback_img = r['images'][0]
            break

    # 检查 Round 1 结果
    for res in results_1:
        if res.get('type') == 'text':
            p_vals = extract_all_prices([res])
            if p_vals:
                res['price_val'] = p_vals[0]
                all_candidates.append(res) # 存入候选池
                if p_vals[0] <= budget_limit:
                    # 完美命中！
                    if not res.get('img') and fallback_img: res['img'] = fallback_img
                    return res, f"约 {p_vals[0]}元", current_kw

    # --- Round 2: 策略修正 (Feedback Loop) ---
    # 只有当找到了价格但都超标时，才触发修正
    avg_price = np.mean([c['price_val'] for c in all_candidates]) if all_candidates else 0
    
    if avg_price > budget_limit:
        print(f"       💸 价格超标 (均价 {int(avg_price)} > 预算 {int(budget_limit/1.2)})，触发修正...")
        
        correction_prompt = f"""
        原策略 "{current_kw}" 市场均价约 {int(avg_price)}元，超预算。
        请推荐一个 **同品类但更便宜** 的具体型号（平替）。
        只输出关键词，不要解释。
        """
        new_kw = profiler_agent.run(correction_prompt).strip()
        print(f"       🔄 军师修正策略为: {new_kw}")
        current_kw = new_kw # 更新关键词
        
        # 执行第二次搜索
        results_2 = hunter.search_raw(f"{new_kw} 价格 RMB")
        
        # 提取第二轮的图片
        for r in results_2:
            if r.get('images'): 
                fallback_img = r['images'][0]
                break
        
        # 检查 Round 2 结果
        for res in results_2:
            if res.get('type') == 'text':
                p_vals = extract_all_prices([res])
                if p_vals:
                    res['price_val'] = p_vals[0]
                    all_candidates.append(res) 
                    if p_vals[0] <= budget_limit:
                        if not res.get('img') and fallback_img: res['img'] = fallback_img
                        return res, f"约 {p_vals[0]}元 (平替)", current_kw

    # --- Round 3: 智能兜底 ---
    # 既然都没完美匹配，就从候选池里挑一个最便宜的，或者直接硬取第一个
    
    best_fallback = None
    status_msg = "暂无报价"
    
    if all_candidates:
        # 按价格排序，取最低的那个
        best_fallback = sorted(all_candidates, key=lambda x: x['price_val'])[0]
        status_msg = f"约 {best_fallback['price_val']}元 (⚠️超预算)"
    elif results_1:
        # 连价格都没搜到，就硬取第一条文本结果
        for res in results_1:
            if res.get('type') == 'text':
                best_fallback = res
                break
    
    if best_fallback:
        # 补图逻辑
        if not best_fallback.get('img') and fallback_img:
            best_fallback['img'] = fallback_img
        return best_fallback, status_msg, current_kw
        
    return None, "搜索失败", current_kw


In [ ]:
# ==========================================
# 🚀 主执行流程
# ==========================================

if not user_input_data:
    print("❌ 未加载用户数据")
else:
    # 0. 解析预算
    budget_min, budget_max = parse_budget_range(user_input_data.get('预算', ''))
    budget_limit = budget_max * 1.2 # 允许超标 20%
    print(f"\n💰 预算红线: {budget_max}元 (容忍至 {budget_limit}元)")

    # 1. 军师制定策略
    profile_text = "\n".join([f"- {k}: {v if v else '未知/不限'}" for k, v in user_input_data.items()])
    print(f"\n🚀 任务启动...\n{'-'*40}")
    print("\n🧠 [1/3] 军师正在制定初步策略...")
    search_strategy = profiler_agent.run(f"请根据以下用户画像制定搜索策略：\n\n{profile_text}")
    print(f"📝 策略: \n{search_strategy}")

    # 2. 准备循环
    keywords = [k.strip() for k in search_strategy.replace("，", ",").replace("\n", ",").split(',') if k.strip()]
    final_items = []
    hunter = BatchSearchTool()

    print(f"\n🔄 进入处理流程 (共 {len(keywords)} 个商品)...")

    for index, kw in enumerate(keywords):
        print(f"\n    👉 [商品 {index+1}/{len(keywords)}] 正在处理: {kw}")
        
        # 调用智能搜索函数
        valid_result, price_status, final_kw = find_best_product(hunter, profiler_agent, kw, budget_limit)
        
        if not valid_result:
            print("       ❌ 彻底无数据，跳过。")
            continue
            
        # === 生成文案 ===
        product_name = valid_result.get('title', final_kw)
        
        print(f"       ✍️ 正在撰写文案: {product_name[:15]}...")
        pitch_prompt = f"""
        商品：{product_name}
        价格：{price_status}
        卖点片段：{valid_result.get('content', '')[:200]}...
        
        请写一句30字以内的种草文案。
        """
        pitch = pitcher_agent.run(pitch_prompt)
        
        # 整理最终数据
        final_items.append({
            "name": final_kw, 
            "title_full": product_name,
            "price": price_status,
            "desc": pitch.replace("\n", " ").strip(),
            "img": valid_result.get('img', ''),
            "link": valid_result.get('url', '')
        })
        print(f"       ✅ 已收录 (状态: {price_status})")


### 第6部分：输出礼物计划

In [ ]:
# --- 4. 渲染与保存 (保持不变) ---
print(f"\n💾 正在生成最终报告...")
if not final_items:
    final_md = "很抱歉，网络搜索似乎出现了问题，未能获取到任何商品信息。"
else:
    table_header = "| 🎁 礼物名称 | 💰 价格 | ✨ 种草理由 | 🖼️ 图片/链接 |\n| :--- | :--- | :--- | :--- |\n"
    table_rows = []
    for item in final_items:
        name = item['name'].replace("|", "/")
        price = item['price']
        desc = item['desc'].replace("|", "/")
        link = item['link']
        img = item['img']
        
        if img and img.startswith("http"):
            media = f"[![图]({img})]({link})"
        else:
            media = f"[点击购买]({link})"
        
        table_rows.append(f"| [{name}]({link}) | {price} | {desc} | {media} |")
    final_md = table_header + "\n".join(table_rows)
filename = "outputs/gift_plan_output.md"
with open(filename, "w", encoding="utf-8") as f:
    f.write(final_md)
print(f"🎉 任务完成！文件已保存: {os.path.abspath(filename)}")

### 第7部分：总结与展望

#### 实现的功能
- 基于用户输入的个人信息，生成符合预算的礼物建议
- 支持用户自定义预算范围、节日、个人喜好等
- 利用搜索引擎获取最新的商品信息和价格
- 提供可视化的建议结果展示
#### 遇到的挑战
- 如何解决大模型的幻觉问题
- 如何解决上下文过长导致提取失败
- 如何设计合理的提示词，优化大模型的输出

#### 未来改进方向
- 前端交互：新增前端页面，提供更好的用户交互体验
- 数据源增强：接入百度或BigGo的MCP Server，获取更精准的实时价格和商品信息
- 丰富选项：增加更多的个人喜好选项，如喜欢的商品类型、品牌等
